# This is a notebook to evaluate trained  TRPO agent in SurfRecon environment

In [1]:
%load_ext autoreload
%autoreload 2
%env MKL_NUM_THREADS=1
%env OMP_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env MKL_DEBUG_CPU_TYPE=5

env: MKL_NUM_THREADS=1
env: OMP_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: MKL_DEBUG_CPU_TYPE=5


In [2]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
seed = 30
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2024-11-26 12:26:58.111548: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 12:26:58.324341: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 12:26:58.339450: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 12:26:58.339507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

# Set up the environment

In [3]:
import sys
import os
# 强制加入正确的路径
sys.path.insert(0, '/root/apps/surfrecon')

import gym
from surface_seg.envs.surfrecon_env import MCSEnv
from surface_seg.utils.callback_new import Callback
from tensorforce.execution import Runner
import gym.wrappers
import numpy as np
import tensorforce
import copy

In [4]:
timesteps = 500
thermal_threshold = 3
num_parallel = 32
# Substitute your own path to save file during evaluation
save_dir = './result_multi_env/experiments/test_surfrecon_eval'

In [5]:
def setup_env(recording=True, structure=None, structure_idx=None):
    
    # Set up gym
    MCS_gym = MCSEnv(observation_fingerprints=True, 
                     observation_forces=True,
                     permute_seed=42, 
                     save_dir = save_dir,
                     timesteps = timesteps,
                     thermal_threshold = thermal_threshold,
                     save_every_min = 1,
                     save_every = 50,
                     step_size = 0.1,                    
                    )
    
    if recording:
    # Wrap the gym to provide video rendering every 50 steps
        MCS_gym = gym.wrappers.Monitor(MCS_gym, 
                                         os.path.join(save_dir, 'vid'), 
                                         force=True,
                                        video_callable = lambda episode_id: (episode_id+1)%50==0) #every 50, starting at 51
    
    #Convert gym to tensorforce environment
    env = tensorforce.environments.OpenAIGym(MCS_gym,
                                         max_episode_timesteps=timesteps,
                                         visualize=False)
    
    return env


"""
Create a environment for checking the intial energy and thermal energy
"""
env = setup_env().environment.env
print('initial energy', env.initial_energy)
print('thermal energy', env.thermal_energy)
n =thermal_threshold
print('%dKT' %n, n * env.thermal_energy)

initial energy 2.4803847437851605
thermal energy 0.413616
3KT 1.240848


/root/anaconda3/envs/catgym_AgPd/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


# Set up the gym and agent in tensorforce

In [6]:
from tensorforce.agents import Agent
# Substitute your own path to saved trained agent
saved_agent_dir = os.path.join('../../surface_seg/pretrained/', 'saved_agent')
agent = Agent.load(saved_agent_dir)

2024-11-26 12:27:00.445667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 12:27:00.445762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 12:27:00.445775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldl(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldl(fn, elems))
INFO:tensorflow:Graph was finalized.


2024-11-26 12:27:03.844241: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 12:27:03.844425: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 12:27:03.844552: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 12:27:03.845286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-26 12:27:03.845314: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1594] Could not ident

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Restoring parameters from ../../surface_seg/pretrained/saved_agent/agent


In [7]:
# Check agent specifications
print(agent.spec)

OrderedDict([('agent', 'ppo'), ('states', {'TS': {'type': 'float', 'shape': [1]}, 'energy': {'type': 'float', 'shape': [1]}, 'fingerprints': {'type': 'float', 'shape': [112]}, 'forces': {'type': 'float', 'shape': [24]}, 'positions': {'type': 'float', 'shape': [24]}}), ('actions', {'action_type': {'type': 'int', 'shape': [], 'num_values': 4}, 'atom_selection': {'type': 'int', 'shape': [], 'num_values': 8}, 'movement': {'type': 'float', 'shape': [1, 3], 'min_value': -0.10000000149011612, 'max_value': 0.10000000149011612}}), ('max_episode_timesteps', 500), ('batch_size', 1), ('network', 'auto'), ('use_beta_distribution', True), ('memory', 50000), ('update_frequency', None), ('learning_rate', 0.001), ('subsampling_fraction', 0.33), ('optimization_steps', 10), ('likelihood_ratio_clipping', 0.2), ('discount', 0.99), ('estimate_terminal', False), ('critic_network', None), ('critic_optimizer', None), ('preprocessing', None), ('exploration', {'type': 'decaying', 'unit': 'timesteps', 'decay': 'e

# Run the trained DRL agent in parallel (multiple environments)

In [8]:
num_episode = 1000

callback = Callback(num_episode, save_dir).episode_finish

runner = Runner( 
    agent=agent,
    environments=[setup_env(recording=False) for _ in range(num_parallel)],
    max_episode_timesteps=timesteps,
    remote='multiprocessing',
)

runner.run(num_episodes=num_episode, callback=callback, callback_episode_frequency=1)
runner.close()

Episodes: 100%|████████| 1000/1000 [1:38:07, reward=0.00, ts/ep=500, sec/ep=186.12, ms/ts=372.2, agent=3.0%, comm=91.0%]
